<a href="https://colab.research.google.com/github/Nianguang-Zhao/Stock-tracker/blob/main/stock_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

# Define the ticker symbols
tickers = ['QQQ', 'SPY', "AAPL", "GOOGL", "META","AMZN","NVDA","MSFT","TSLA"]

# Download data from Yahoo Finance
data = yf.download(tickers, start='2024-12-31', progress=False)

In [84]:
# Extract only the 'Close' prices
close_prices = data['Close']

# Calculate daily returns
daily_returns = close_prices.pct_change()

# Get latest date
last_date = close_prices.index.max()

# Determine start dates for each return period
weekly_start = close_prices.index[-6]  # 5 trading days ago
monthly_start = last_date.to_period('M').to_timestamp('D')
monthly_start = close_prices[close_prices.index >= monthly_start].index[0]
ytd_start = last_date.to_period('Y').to_timestamp('D')
ytd_start = close_prices[close_prices.index >= ytd_start].index[0]

In [85]:
# Calculate cumulative returns (normalized to start from 0%)
def compute_normalized_returns(start_date):
    sub_returns = daily_returns[daily_returns.index >= start_date]
    cum_returns = (1 + sub_returns).cumprod() - 1
    return cum_returns - cum_returns.iloc[0]  # Normalize to start from 0

weekly_cumulative = compute_normalized_returns(weekly_start)
monthly_cumulative = compute_normalized_returns(monthly_start)
ytd_cumulative = compute_normalized_returns(ytd_start)

# Prepare the interactive plot
fig = go.Figure()

# Add traces for each ticker and return period
for ticker in tickers:
    # Weekly
    fig.add_trace(go.Scatter(x=weekly_cumulative.index,
                             y=weekly_cumulative[ticker] * 100,
                             name=f'{ticker} Weekly Return',
                             visible=True,
                             mode='lines'))
    fig.add_trace(go.Scatter(x=monthly_cumulative.index,
                             y=monthly_cumulative[ticker] * 100,
                             name=f'{ticker} Monthly Return',
                             visible=False,
                             mode='lines'))
    fig.add_trace(go.Scatter(x=ytd_cumulative.index,
                             y=ytd_cumulative[ticker] * 100,
                             name=f'{ticker} YTD Return',
                             visible=False,
                             mode='lines'))

# Add end-point annotations
annotations = []
for i, (data, label) in enumerate(zip([weekly_cumulative, monthly_cumulative, ytd_cumulative],
                                      ['Weekly Return', 'Monthly Return', 'YTD Return'])):
    for j, ticker in enumerate(tickers):
        value = data[ticker].iloc[-1] * 100
        annotations.append(dict(
            x=data.index[-1],
            y=value,
            xref='x',
            yref='y',
            text=f'{ticker}: {value:.2f}%',
            showarrow=True,
            arrowhead=1,
            ax=40,
            ay=-10,
            font=dict(color='black'),
            visible=(i == 0)
        ))

# Dropdown buttons
buttons = []
for i, label in enumerate(['Weekly Return', 'Monthly Return', 'YTD Return']):
    visibility = [False] * len(fig.data)
    annotation_vis = [False] * len(annotations)
    for j in range(len(tickers)):
        visibility[i + 3 * j] = True
        annotation_vis[i * len(tickers) + j] = True
    buttons.append(dict(label=label,
                        method='update',
                        args=[{'visible': visibility},
                              {'title': f'{label} Comparison', 'annotations': [
                                  {**a, 'visible': vis} for a, vis in zip(annotations, annotation_vis)
                              ]}]))

# Update layout
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.1,
        y=1.15,
        xanchor='left',
        yanchor='top'
    )],
    title='Weekly Return Comparison',
    xaxis_title='Date',
    yaxis_title='Return (%)',
    legend_title='Ticker',
    height=600,
    yaxis_tickformat='.2f',
    annotations=[a for a in annotations if a['visible']]
)

# Show the figure
fig.show()